In [166]:
!pip install -U --no-deps "feedparser>=6.0.8"
!pip install --no-deps pygooglenews==0.1.2
!pip install -U --no-deps "dateparser>=1.0.0"
!pip install tzlocal
!pip3 install newspaper3k

  Using cached pygooglenews-0.1.2-py3-none-any.whl (10 kB)


In [246]:
import re
import numpy as np
from newspaper import Article
import pandas as pd
from collections import Counter
from pygooglenews import GoogleNews
from dateutil import rrule
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

import itertools
import time
import math

import pandas as pd
from multiprocessing import Pool
from newspaper import Article
from multiprocessing import cpu_count
import time
from collections import Counter
from langdetect import detect
from tqdm import tqdm
import sys
from collections import Counter
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

/var/folders/zn/scrcm7bn6xl2n15flz1swk6m0000gn/T/ipykernel_23303/1604985006.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [50]:
my_dir = '/Users/irinaevdokimova/Documents/ai_house_fake_news_detection/Week_2_Linear_and_Recurrent_models/homework'

### 1- parse https://rusdisinfo.voxukraine.org/

In [51]:
import pandas as pd

fake = pd.read_csv(my_dir + '/Disinfo narratives base eng.csv')

In [59]:
fake_unique = fake.drop_duplicates(subset=['Debunking'], keep='first')

In [220]:
fake_unique.dtypes

id                   int64
Narrative           object
Fakes               object
Disinfo_cases_en    object
Date                object
Countries           object
Media               object
Link                object
Debunking           object
dtype: object

### 2 - parse google news

In [176]:
news_count= 50
dateTo = datetime.now()
dateFrom = dateTo - relativedelta(months=2)
interval = 30
count = math.ceil((dateTo - dateFrom).days/interval)

dates = list(rrule.rrule(rrule.DAILY, dtstart=dateFrom, count=count, interval=interval))
dates.reverse()
dates

[datetime.datetime(2023, 1, 30, 22, 14, 57),
 datetime.datetime(2022, 12, 31, 22, 14, 57),
 datetime.datetime(2022, 12, 1, 22, 14, 57)]

#### 2.1 - collect urls

In [180]:
lng = 'EN'
lang=lng.lower()

In [187]:
query ='ukraine'

In [189]:
docs = {}
for key in tqdm(range(0,200)):
     
    for dt_from in dates:
        gn = GoogleNews(lang=lang)
        dt_from, dt_to = str(dt_from).split(' ')[0], str(dt_from + relativedelta(days=interval)).split(' ')[0]
        try:
            search = gn.search('allintitle:' + query, from_=dt_from, to_=dt_to)
            if len(search['entries']) == 0:
                search = gn.search(query, from_=dt_from, to_=dt_to)
        except:
            try:
                gn = GoogleNews(lang=lang)
                search = gn.search('allintitle:' + query, from_=dt_from, to_=dt_to)
                if len(search['entries']) == 0:
                    search = gn.search(query, from_=dt_from, to_=dt_to)
            except:
                continue
                
        if key not in docs.keys():
            docs[key] = []
        try:
            docs[key] += [case['link'] for i, case in enumerate(search['entries'])][:5]
            if len(docs[key]) >= news_count:
                docs[key] = docs[key][:news_count]
                break
            #print(key, len(docs[key]))
        except:
            continue

100%|█████████████████████████████████████████| 200/200 [04:47<00:00,  1.44s/it]


In [193]:
df = pd.DataFrame(
    columns=['Url'],
    data = [(url) for key, urls in docs.items() for url in urls]
)

In [202]:
df = df.iloc[0:100]

#### 2.2 Scrape google news

In [203]:
def scrape(lst):
    try:
        article = Article(lst[-1], fetch_images=False)
        article.download()
        article.parse()
        return [lst[0], article.title + '. ' + article.text]
    except:
        return [lst[0], lst[1]]

In [204]:
texts = []
for i in tqdm(df.values.tolist()):
    scrapped =  scrape(i)
    texts.append(scrapped)

100%|█████████████████████████████████████████| 100/100 [00:59<00:00,  1.67it/s]


In [211]:
only_text = [j[1] for j in texts]

#### 3 - create train data set

In [274]:
train_df_good = pd.DataFrame()
train_df_good['text'] = only_text
train_df_good['label'] = 1

In [275]:
train_df_bad = pd.DataFrame()
train_df_bad['text'] = fake_unique['Debunking']
train_df_bad['label'] = 0

In [276]:
train_overall = train_df_good.append(train_df_bad)

/var/folders/zn/scrcm7bn6xl2n15flz1swk6m0000gn/T/ipykernel_23303/931660309.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_overall = train_df_good.append(train_df_bad)


In [277]:
train_overall['text']

0      Ukraine war: Russia planning 24 February offen...
1      Russia is preparing for 'maximum escalation' i...
2      Fighting ramps up in eastern Ukraine in 'devas...
3      Russia-Ukraine War Ukraine Carries Out Wide An...
4      Russia-Ukraine war at a glance: what we know o...
                             ...                        
484    <p>Most of the messages claiming ostensible Uk...
514    <p>Ukraine did not steal Russian gas. Similar ...
545    <p>Representatives of the pseudo-republics, th...
578    <p>Russian propaganda was spread in the church...
586    <p>Russia’s accusations of supporting the war ...
Name: text, Length: 179, dtype: object

#### 3.1 - preprocess text for naive bayes

In [289]:
#Removes the punctuation from the real_df data frame.
train_overall['text'] = train_overall['text'].str.replace('\n','')
train_overall['text'] = train_overall['text'].str.replace('<p>','')
train_overall['text'] = train_overall['text'].apply(lambda x: re.sub(r'[^\w\s]','',str(x)))
#Removes the numerals from the real_df data frame.
train_overall['text'] = train_overall['text'].str.replace('\d+','')
#Removes the quotation marks from the real_df data frame.
train_overall['text'] = train_overall['text'].apply(lambda x: re.sub(r'\"', '',str(x)))
#Lower case for all text.
train_overall['text'] = train_overall['text'].str.lower()

/var/folders/zn/scrcm7bn6xl2n15flz1swk6m0000gn/T/ipykernel_23303/2096729971.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train_overall['text'] = train_overall['text'].str.replace('\d+','')


In [290]:
train_overall = train_overall.sample(frac=1)

In [291]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [292]:
X = train_overall['text'].values
y = train_overall['label'].values
vector = CountVectorizer(stop_words = 'english')
#Fits/transforms the count vectorizor on the X variable
X = vector.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 30)
bayes_pred = MultinomialNB()
bayes_pred.fit(X_train, y_train)
#Creates predictions based on the model
predictions = bayes_pred.predict(X_test)

In [293]:
print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))

[[25  0]
 [ 0 20]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00        20

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [294]:
name = 'bayes_model.pkl'
pickle.dump(bayes_pred, open(name, 'wb'))